# PRACTICA GUIADA 1: Métricas de evaluación de modelos

## 1. Introducción

El objetivo de esta práctica es analizar en la práctica las medidas de evaluación para modelos de clasificación mencionadas.

Para ello trabajaremos tratando de predecir la probabilidad de que un empleado deje la empresa. Para ello se dispone de un dataset 

Los campos incluidos son:

1. Última evaluación
2. Cantidad de proyectos en los que trabajó
3. Promedio de horas mensuales trabajadas
4. Tiempo en la compañía
5. Sufrió un accidente de trabajo
6. Tuvo una promoción en el último año
7. Nivel salarial

El objetivo, entonces, es predecir la probabilidad de que $P(left=1 | X)$ 

## 2. Métricas de evaluación para problemas de clasificación

Como de costumbre, importamos los datos y el dataset

In [1]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Data/HR_comma_sep.csv')
df.sample(10)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
598,0.37,0.46,2,157,3,0,1,0,IT,low
12137,0.42,0.48,2,129,3,0,1,0,management,low
1661,0.36,0.56,2,140,3,0,1,0,technical,medium
1225,0.39,0.46,2,127,3,0,1,0,management,low
13395,0.84,0.75,4,135,3,0,0,1,RandD,medium
8039,0.56,0.84,4,143,4,0,0,0,sales,medium
12067,0.90,0.98,4,264,6,0,1,0,product_mng,medium
14288,0.40,0.54,2,137,3,0,1,0,marketing,medium
8448,0.99,0.79,4,213,3,0,0,0,support,low
6,0.10,0.77,6,247,4,0,1,0,sales,low


Armamos la matriz de predictores ($X$) y el target ($y$)

In [3]:
train_cols = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 
              'time_spend_company', 'Work_accident', 'promotion_last_5years']
X = df[train_cols]
y = df['left']

Hacemos el split entre train y test:

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### 2.1 Entrenando un primer clasificador

Como primer paso (y para mantener el problema simple) comencemos entrenando una regresión logística.

In [5]:
clf = LogisticRegression(C=1e10)
clf.fit(X_train, y_train)

LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [6]:
y_pred = clf.predict(X_test)

### 2.2 Métricas: Accuracy

Como recordarán, el accuracy se calcula como la proporción samples correctamente clasificados sobre el total de samples.

In [7]:
from sklearn.metrics import accuracy_score
print('Accuracy=', accuracy_score(y_test, y_pred))

Accuracy= 0.757171717172


Es decir, que en este caso, encontramos que el 76% delos casos -en el test set- han sido correctamente clasificados.

Ahora bien, ¿qué tan bueno es este clasificador? ¿Qué significa que podamos clasificar correctamente a esta proporción de casos?

Una primera forma de comenzar a responder esta pregunta es comparar la performance con un clasificador bien simple y (casi) trivial: se lo suele llamar "clasificador nulo" y consiste simplemente en predecir solamente teniendo en cuenta la clase más frecuente.

In [8]:
y_test.value_counts()

0    3769
1    1181
Name: left, dtype: int64

In [9]:
y_test.mean()

0.2385858585858586

Es decir que 23% de los casos en el train-set son 1, es decir, se irán de la empresa. Por ende, la proporción de 0 (es decir, casos que no se van de la empresa) será:

In [10]:
1.0 - y_test.mean()

0.7614141414141414

Nuestro modelo simple, entonces, haría predicciones siempre igual a cero. Si realizáramos las predicciones en función de este dato... ¿qué accuracy esperaríamos obtener...? En efecto, esperaríamos obtener una accuracy cercana al 76%. Es decir, esperaríamos estar en lo correcto (sin ninguna otra informacion) en un 76% de los casos.

De esta forma, pareciera que el modelo de regresión logística no es tan bueno: no parece mejorar demasiado respecto al modelo simple. Si solamente consideráramos el accuracy podríamos habernos equivocado en la evaluación de nuestro modelo. Por eso suele ser útil considerar otras métricas de evaluación. 

### 2.3 Métricas: Confusion Matrix

Si bien hemos estado trabajando con este insumo, hasta aquí lo veníamos haciendo de forma intuitiva. Tratemos de entender mejor qué es una matriz de confusión.

Básicamente, es una tabla de contingencia que tabula la distribución de los casos analizados en función de su valor real ("observado") y su valor estimado por el modelo ("predicho"). 

En `confusion_matrix` es importante recordar que el primer arugmento corresponde a los valores observados y el segundo a los valores predichos:

In [11]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

[[3465  304]
 [ 898  283]]


En las filas están representados los datos observados (`y_test`). En las columnas se representan los datos predichos por el modelo (`y_pred`).

** Matriz de confusión **

|                        | Pred Stay ($y\_pred=0$)| Pred Left ($y\_pred=1$)| Total|
| :--------------------  |:----------------------:| :---------------------:|-----:|
| Obs Stay ($y\_test=0$) | 3465                   | 304                    |3769  |
| Obs Left ($y\_test=1$) | 898                    | 283                    |1181  | 
| Total                  | 4363                   | 587                    |N=4950|
     
Ahora bien, cada casilla aporta información sobre la performace del clasificador:

* **True Positives (TP):** hemos predicho correctamente que el empleado se va (295)
* **True Negatives (TN):** hemos predicho correctamente que el empleado se queda (3497)
* **False Positives (FP):** hemos predicho que el empleado se iba pero se queda (265)
* **False Negatives (FN):** hemos predicho que el empleado se queda pero se va (893)

Asignemos a variables estos casos para realizar algunos cálculos:

In [12]:
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

### 2.4 Métricas computadas desde la matriz de confusión: Accuracy

In [13]:
print((TP + TN) / float(TP + TN + FP + FN))
print(accuracy_score(y_test, y_pred))

0.757171717172
0.757171717172


### 2.5 Métricas computadas desde la matriz de confusión: Classification Error

Es, básicamente, el complemento de accuracy. Cuantifica el error total cometido por el clasificador:

In [14]:
class_error = (FP + FN) / float(TP + TN + FP + FN)

print(class_error)
print(1 - accuracy_score(y_test, y_pred))

0.242828282828
0.242828282828


### 2.6 Métricas computadas desde la matriz de confusión: Sensitivity (o recall)

Mide la capacidad (qué tan "sensible" es) del modelo de detectar los verdaderos positivos (TP) sobre todos los casos que son positivos (FN + TP). En nuestro ejemplo: del total de personas que se van, ¿cuántas logra clasificar correctamente el modelo?

In [15]:
from sklearn.metrics import recall_score
sensitivity = TP / float(FN + TP)

print(sensitivity)
print(recall_score(y_test, y_pred))

0.239627434378
0.239627434378


### 2.7 Métricas computadas desde la matriz de confusión: Specificity

Mide la capacidad de detectar los "verdaderos negativos" (TN) sobre el total de casos que son negativos (TN + FP). ¿Qué tan específico o selectivo es el modelo al predecir las instancias positivas?

In [16]:
specificity = TN / (TN + FP)

print(specificity)

0.919342000531


Nuestro modelo parece ser muy específico y poco sensitivo.

### 2.8 Métricas computadas desde la matriz de confusión: Precision

Mide qué tan "preciso" es el clasificador al predecir las instancias positivas. Es decir, cuando el clasificador predice un valor positivo... ¿qué tan frecuentemente es esta predicción correcta?

In [17]:
from sklearn.metrics import precision_score

precision = TP / float(TP + FP)

print(precision)
print(precision_score(y_test, y_pred))

0.482112436116
0.482112436116


### 2.9 Métricas computadas desde la matriz de confusión: F1-Score

Es un promedio armónimo entre precision y recall.

In [18]:
from sklearn.metrics import f1_score

f1 = 2*((precision*sensitivity)/(precision+sensitivity))

print(f1)
print(f1_score(y_test,y_pred))

0.320135746606
0.320135746606


### 3. Conclusiones

La matriz de confusión brinda un panorama más completo sobre la performance del clasificador. 

¿Sobre qué métricas habría que focalizarse? Obviamente, depende del problema, del objetivo.

* **Ejemplo - filtro de SPAM:** pareciera que los FN son más aceptables (spam entra en la casilla) que los FP (un mail útil es filtrado como SPAM).


* **Ejemplo - detector de fraudes:** en este caso, pareciera ser preferible tolerar FP (transacciones que NO son falsas como falsas) que dejar pasar TP (transacciones fraudulentas que no son detectadas). Sería preferible minimizar sensitivity.

## 5. Bonus: haciendo validación cruzada con otras métricas

Supongamos que queremos estimar el error de generalización de nuestro modelo de regresión pero usando cross-validation. Podemos usar la función `cross_val_score` cambiando la métrica a evaluar:

In [19]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

kf = StratifiedKFold(n_splits=10, shuffle = True)

print('F1-CV=', np.mean(cross_val_score(clf, X, y, cv=kf, scoring='f1')))
print('Recall-CV=', np.mean(cross_val_score(clf, X, y, cv=kf, scoring='recall')))
print('Specificity-CV=', np.mean(cross_val_score(clf, X, y, cv=kf, scoring='precision')))

F1-CV= 0.339004143997
Recall-CV= 0.252585168145
Specificity-CV= 0.517608845537
